In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 1000)
pd.set_option('max_columns',100)

# Clean Dataset

# Clean Order Data

In [ ]:
def rename_categories(prefix, current_name):
    return f'{prefix}:{current_name}'

def test_rename_categories():
    obs = rename_categories(0,'It_Worked!')
    exp = '0:It_Worked!'
    print(obs)
    print(exp)
    assert obs == exp

test_rename_categories()

In [ ]:
order = pd.read_csv('data/order.csv')
order['orderdate'] = pd.to_datetime(order['orderdate'])

In [ ]:
order.shape

#### Clean Order Data

In [ ]:
def clean_96_overlap(prd_cat1, prd_cat2):
    if prd_cat2 == 96:
        prd_cat1 = 7
    return prd_cat1

def test_clean_96_overlap():
    in_data = [
        [0,1],
        [3,96],
        [7,96]
    ]
    obs = [clean_96_overlap(x[0], x[1]) for x in in_data]
    exp = [0,7,7]
    assert obs == exp

test_clean_96_overlap()

def prod_cat2_nan_fill(prd_cat1, prd_cat2):
    if np.isnan(prd_cat2):
        return -1*prd_cat1
    else:
        return prd_cat2

def test_prod_cat2_nan_fill():
    in_data = [
        [1,np.nan],
        [2,np.nan],
        [3,199],
        [1,2]
    ]
    obs = [prod_cat2_nan_fill(x[0], x[1]) for x in in_data]
    exp = [-1,-2, 199, 2]
    assert obs == exp, f'obs:{obs}, exp:{exp}'

test_prod_cat2_nan_fill()

In [ ]:
order['prodcat2'] = order[['prodcat1', 'prodcat2']].apply(
    lambda row: prod_cat2_nan_fill(row['prodcat1'], row['prodcat2']), axis=1
)

In [ ]:
order['prodcat1'] = order[['prodcat1', 'prodcat2']].apply(
    lambda row: clean_96_overlap(row['prodcat1'], row['prodcat2']), axis=1
)

#### Get Dummies for prodcat2 and prodcat1

In [ ]:
order['prodcat1'] = order[['prodcat1']].apply(lambda row: rename_categories('P1',row[0]), axis=1)
order['prodcat2'] = order[['prodcat2']].apply(lambda row: rename_categories('P2',row[0]), axis=1)

In [ ]:
p1_dummies = pd.get_dummies(order['prodcat1'])
p2_dummies = pd.get_dummies(order['prodcat2'])

In [ ]:
pcat_columns = list(p1_dummies.columns) + list(p2_dummies.columns)

In [ ]:
order_expanded = pd.concat([order, p1_dummies, p2_dummies], axis=1)

In [ ]:
order_expanded.head()

In [ ]:
# Create Dictionary for Aggregation
order_agg_dict = {}
for pcat_column in pcat_columns:
    order_agg_dict[pcat_column] = 'sum'
order_agg_dict['orderdate'] = 'min'

In [ ]:
order_totals = order_expanded.groupby(['ordno', 'custno'], as_index=False).agg(order_agg_dict)

In [ ]:
order_totals['ordermonth'] = order_totals['orderdate'].dt.month

In [ ]:
assert order_totals.shape[0] == len(order_totals['ordno'].unique())

# Create Modeling Datsets

In [ ]:
modeling_dataset = order_totals[
    ['ordno', 'custno', 'orderdate', 'ordermonth','P1:1.0','P1:2.0','P1:3.0','P1:4.0','P1:5.0','P1:7.0']
].sort_values('ordno').copy()

In [ ]:
# Confirm Unique at Order Level
assert (modeling_dataset[['ordno','custno']].drop_duplicates().shape[0] 
        == modeling_dataset[['ordno']].drop_duplicates().shape[0])

In [ ]:
merged_orders = modeling_dataset.merge(
    order_totals, on='custno', how='left',
    suffixes=('_current', '_previous')
)

In [ ]:
all_prev_orders = merged_orders[merged_orders['orderdate_current']> merged_orders['orderdate_previous']].copy()

In [ ]:
all_prev_orders['days_before'] = (
    all_prev_orders['orderdate_current'] - all_prev_orders['orderdate_previous']
).astype('timedelta64[s]')/3600/24

In [ ]:
all_prev_orders_clean = all_prev_orders[
    [
        'ordno_current', 'days_before',
        'P2:-1.0', 'P2:-7.0', 'P2:10.0', 'P2:100.0', 'P2:101.0', 'P2:102.0', 'P2:103.0', 'P2:104.0', 'P2:105.0',
        'P2:106.0', 'P2:107.0', 'P2:108.0', 'P2:109.0', 'P2:11.0', 'P2:110.0', 'P2:111.0', 'P2:112.0', 'P2:113.0',
        'P2:114.0', 'P2:115.0', 'P2:116.0', 'P2:117.0', 'P2:118.0', 'P2:119.0', 'P2:12.0', 'P2:120.0', 'P2:121.0',
        'P2:122.0', 'P2:123.0', 'P2:124.0', 'P2:125.0', 'P2:126.0', 'P2:127.0', 'P2:128.0', 'P2:129.0', 'P2:13.0',
        'P2:130.0', 'P2:131.0', 'P2:132.0', 'P2:133.0', 'P2:134.0', 'P2:135.0', 'P2:136.0', 'P2:137.0', 'P2:138.0',
        'P2:139.0', 'P2:14.0', 'P2:140.0', 'P2:141.0', 'P2:142.0', 'P2:143.0', 'P2:144.0', 'P2:145.0', 'P2:146.0',
        'P2:147.0', 'P2:148.0', 'P2:149.0', 'P2:15.0', 'P2:150.0', 'P2:151.0', 'P2:152.0', 'P2:153.0', 'P2:154.0',
        'P2:155.0', 'P2:156.0', 'P2:157.0', 'P2:158.0', 'P2:159.0', 'P2:16.0', 'P2:160.0', 'P2:161.0', 'P2:162.0',
        'P2:164.0', 'P2:165.0', 'P2:166.0', 'P2:167.0', 'P2:168.0', 'P2:169.0', 'P2:17.0', 'P2:170.0', 'P2:171.0',
        'P2:172.0', 'P2:173.0', 'P2:174.0', 'P2:175.0', 'P2:176.0', 'P2:177.0', 'P2:178.0', 'P2:179.0', 'P2:18.0',
        'P2:180.0', 'P2:181.0', 'P2:182.0', 'P2:183.0', 'P2:184.0', 'P2:185.0', 'P2:186.0', 'P2:187.0', 'P2:188.0',
        'P2:189.0', 'P2:19.0', 'P2:190.0', 'P2:191.0', 'P2:192.0', 'P2:193.0', 'P2:194.0', 'P2:195.0', 'P2:196.0',
        'P2:197.0', 'P2:198.0', 'P2:199.0', 'P2:2.0', 'P2:20.0', 'P2:200.0', 'P2:201.0', 'P2:202.0', 'P2:203.0',
        'P2:204.0', 'P2:205.0', 'P2:206.0', 'P2:207.0', 'P2:208.0', 'P2:209.0', 'P2:21.0', 'P2:210.0', 'P2:211.0',
        'P2:212.0', 'P2:213.0', 'P2:214.0', 'P2:215.0', 'P2:216.0', 'P2:217.0', 'P2:218.0', 'P2:219.0', 'P2:220.0',
        'P2:221.0', 'P2:222.0', 'P2:223.0', 'P2:224.0', 'P2:225.0', 'P2:226.0', 'P2:227.0', 'P2:228.0', 'P2:229.0',
        'P2:23.0', 'P2:230.0', 'P2:231.0', 'P2:232.0', 'P2:233.0', 'P2:234.0', 'P2:235.0', 'P2:236.0', 'P2:237.0',
        'P2:238.0', 'P2:239.0', 'P2:24.0', 'P2:240.0', 'P2:241.0', 'P2:243.0', 'P2:244.0', 'P2:245.0', 'P2:246.0',
        'P2:247.0', 'P2:248.0', 'P2:249.0', 'P2:25.0', 'P2:250.0', 'P2:251.0', 'P2:252.0', 'P2:253.0', 'P2:255.0',
        'P2:256.0', 'P2:257.0', 'P2:258.0', 'P2:259.0', 'P2:26.0', 'P2:260.0', 'P2:261.0', 'P2:262.0', 'P2:263.0',
        'P2:27.0', 'P2:28.0', 'P2:3.0', 'P2:30.0', 'P2:32.0', 'P2:33.0', 'P2:34.0', 'P2:35.0', 'P2:38.0',
        'P2:39.0', 'P2:4.0', 'P2:40.0', 'P2:41.0', 'P2:42.0', 'P2:43.0', 'P2:44.0', 'P2:45.0', 'P2:46.0',
        'P2:47.0', 'P2:48.0', 'P2:49.0', 'P2:5.0', 'P2:50.0', 'P2:51.0', 'P2:52.0', 'P2:53.0', 'P2:54.0',
        'P2:55.0', 'P2:56.0', 'P2:57.0', 'P2:58.0', 'P2:59.0', 'P2:6.0', 'P2:60.0', 'P2:61.0', 'P2:62.0',
        'P2:63.0', 'P2:64.0', 'P2:65.0', 'P2:66.0', 'P2:67.0', 'P2:69.0', 'P2:7.0', 'P2:70.0', 'P2:71.0',
        'P2:72.0', 'P2:73.0', 'P2:74.0', 'P2:75.0', 'P2:76.0', 'P2:77.0', 'P2:78.0', 'P2:79.0', 'P2:8.0',
        'P2:80.0', 'P2:81.0', 'P2:82.0', 'P2:83.0', 'P2:85.0', 'P2:86.0', 'P2:88.0', 'P2:89.0', 'P2:9.0',
        'P2:90.0', 'P2:91.0', 'P2:92.0', 'P2:93.0', 'P2:94.0', 'P2:95.0', 'P2:96.0', 'P2:97.0', 'P2:98.0',
        'P2:99.0' 
]

].copy()

In [ ]:
def get_months_before(days):
    i = 0
    for max_days in range(30,30*24,30):
        i+=1
        if days<max_days:
            return i
    return -1


In [ ]:
all_prev_orders_clean['months_before'] = all_prev_orders_clean['days_before'].apply(get_months_before)

In [ ]:
len(set(all_prev_orders_clean['ordno_current']))

#### Aggregate to Order and months_before level

In [ ]:
all_prev_order_by_month = all_prev_orders_clean.groupby(
    ['ordno_current','months_before'], as_index=False).sum()

#### Make Empty Dataset Unique at ordno and months_before level

In [ ]:
ordno_df = modeling_dataset[['ordno']].copy()
ordno_df['key'] = 1

months_before_df = pd.DataFrame(np.arange(1,24).reshape(-1,1), columns=['months_before'])
months_before_df['key']=1

ordno_months_before_df = ordno_df.merge(months_before_df, on='key')
del(ordno_months_before_df['key'])

In [ ]:
ordno_months_before_df.head()

#### Make Dataset of Last 24 Months of Data

In [ ]:
previous_orders_by_month = ordno_months_before_df.merge(
    all_prev_order_by_month, 
    left_on=['ordno', 'months_before'],
    right_on=['ordno_current', 'months_before'],
    how='left'
)

In [ ]:
previous_orders_by_month = previous_orders_by_month.fillna(0)

In [ ]:
del(ordno_months_before_df)

## Merge previous_orders_by_month with the dependent variables

In [ ]:
dependent_vars = modeling_dataset.drop(['custno'], axis=1)

In [ ]:
assert dependent_vars.shape[0] == len(set(dependent_vars['ordno']))

In [ ]:
dependent_vars.head()

In [ ]:
final_pd = previous_orders_by_month.merge(dependent_vars, on='ordno',how='left')
final_pd = final_pd[final_pd['months_before']<=14].sort_values(by=['ordno', 'months_before']).copy()

# Write to pickle file

In [ ]:
final_pd.to_pickle('data/final_pd_previous_orders.pkl')

In [ ]:
dependent_vars.to_pickle('data/dependent_vars_previous_orders.pkl')

# 

In [ ]:
assert final_pd[['ordno', 'months_before']].drop_duplicates().shape[0] == final_pd.shape[0]

In [ ]:
final_pd = final_pd.drop(['ordno_current', 'months_before', 'days_before'],axis=1)

In [ ]:
final_pd.head()

In [ ]:
ordno_df.memory_usage()

In [ ]:
for var in dir():
    if isinstance(eval(var), pd.core.frame.DataFrame):
        print(50*'=')
        print(var)
        print(eval(var+'.memory_usage(deep=True).sum()'))

print(alldfs) # df1, df2

In [ ]:
del(p1_dummies)

In [ ]:
del(p2_dummies)

In [ ]:
del(order)

## Split Between Train and Test
X_Orders.shape = (-1, 14, 254)
X_Month.shape = (-1,1)
Y.shape = (-1, 6)

X_Events.shape = ()


#### Get Test Data

In [ ]:
dependent_vars.head()

In [ ]:
final_pd.head()

In [ ]:
assert len(set(dependent_vars['ordno'])) == len(set(final_pd['ordno']))

# Preprocessing

#### Convert Orders From Counts to BInary

In [ ]:
dependent_vars[
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']
] = (dependent_vars[
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']
]>0).astype(int)

## Create Test Dataset

In [ ]:
test_pd = final_pd[final_pd['orderdate']> pd.to_datetime('2018-09-1')]
test_dependent_pd = dependent_vars[dependent_vars['orderdate']> pd.to_datetime('2018-09-1')]

In [ ]:
del(test_pd['orderdate'])

In [ ]:
del(test_pd['ordno'])

In [ ]:
test_X_orders = test_pd.drop(
    ['ordermonth','P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0'],
    axis=1
).values

test_X_orders = test_X_orders.reshape((-1,14, 253))

In [ ]:
test_X_order_month = test_dependent_pd['ordermonth'].values.reshape((-1,1))

In [ ]:
test_Y_order = test_pd[['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']].values

## Create Training Dataset

In [ ]:
train_pd = final_pd[final_pd['orderdate']<= pd.to_datetime('2018-09-1')]
train_dependent_pd = dependent_vars[dependent_vars['orderdate']<= pd.to_datetime('2018-09-1')]

In [ ]:
train_X_orders = train_pd.drop(
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0'],
    axis=1
).values

train_X_orders = train_X_orders.reshape((-1,14, 254))

In [ ]:
test_Y_order

#### Get Train Data

In [ ]:
train_pd = final_pd[final_pd['orderdate']<= pd.to_datetime('2018-09-1')]

In [ ]:
train_X = train_pd.drop(
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0'],
    axis=1
).values

train_X = train_X.reshape((-1,14, 254))

In [ ]:
del(train_pd['orderdate'])

In [ ]:
X = final_pd.drop(
    ['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0'],
    axis=1
).values

In [ ]:
Y = final_pd[['P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0', 'P1:5.0', 'P1:7.0']].shape

In [ ]:
ordno_df = modeling_dataset[['ordno']].copy()
ordno_df['key'] = 1

In [ ]:
all_prev_orders_clean.head()

# Junk Below

#### Previous Order Count

In [ ]:
full_prev_order_count

#### Get Purchase Counts

In [ ]:
def get_order_count_sum(all_previous_orders, min_b4, max_b4):
    relevant_previous_orders = all_previous_orders[
        (all_previous_orders['days_before']>min_b4) 
        & (all_previous_orders['days_before']< max_b4)
    ]
    del(relevant_previous_orders['days_before'])
    
    order_count_sum = relevant_previous_orders.groupby('ordno_current', as_index=False).sum()
    
    # Fix Column Names
    agg_cols = list(order_count_sum.columns)[1:]
    new_cols = [x+f'_{min_b4}_{max_b4}' for x in agg_cols]
    new_cols = ['ordno_current'] + new_cols
    order_count_sum.columns = new_cols

    return order_count_sum

In [ ]:
order_count_sum_0_30 = get_order_count_sum(all_prev_orders_clean,0,30)
order_count_sum_30_350 = get_order_count_sum(all_prev_orders_clean,30,350)
order_count_sum_350_380 = get_order_count_sum(all_prev_orders_clean,350,380)

In [ ]:
order_count_sum_0_30.head()

In [ ]:
assert len(set(order_count_sum_0_30['ordno_current'])) == order_count_sum_0_30.shape[0]
assert len(set(order_count_sum_30_350['ordno_current'])) == order_count_sum_30_350.shape[0]
assert len(set(order_count_sum_350_380['ordno_current'])) == order_count_sum_350_380.shape[0]

assert len(set(modeling_dataset['ordno'])) == modeling_dataset.shape[0]

In [ ]:
modeling_dataset.head()

In [ ]:
modeling_dataset_orders = modeling_dataset.merge(
    order_count_sum_0_30,
    left_on='ordno', right_on='ordno_current',
    how='left'
).merge(
    order_count_sum_30_350,
    left_on='ordno', right_on='ordno_current',
    how='left'
).merge(
    order_count_sum_350_380,
    left_on='ordno', right_on='ordno_current',
    how='left'
)

In [ ]:
# Ensure all Orders are stored
assert modeling_dataset.shape[0]==modeling_dataset_orders.shape[0]

In [ ]:
assert len(set(modeling_dataset_orders['ordno'])) == modeling_dataset_orders.shape[0]

In [ ]:
modeling_dataset_orders.head()

# Clean Online Data
- Aggregate to Session Level

#### Read in Data

In [ ]:
# Read in Data
online = pd.read_csv('data/online.csv')

# Convert to Datetime
online['dt'] = pd.to_datetime(online['dt'])

# Fill in Missing Values with Dummie Value
online['event1'] = online['event1'].fillna(-1)

# Give clear category name for when it's split out to all the columns
online['event1'] = online[['event1']].apply(lambda row: rename_categories('E1',row[0]), axis=1)
online['event2'] = online[['event2']].apply(lambda row: rename_categories('E2',row[0]), axis=1)
online['category'] = online[['category']].apply(lambda row: rename_categories('Cat',row[0]), axis=1)

#### Split Out Events

In [ ]:
e1_dummies = pd.get_dummies(online['event1'])
e2_dummies = pd.get_dummies(online['event2'])
cat_dummies = pd.get_dummies(online['category'])

In [ ]:
event_vars = list(e1_dummies.columns) + list(e2_dummies.columns) + list(cat_dummies.columns)

In [ ]:
online_expanded = pd.concat([online, e1_dummies, e2_dummies, cat_dummies], axis=1)

In [ ]:
online_expanded.head()

In [ ]:
# Create Dictionary for Aggregation
agg_dict = {}
for event_var in event_vars:
    agg_dict[event_var] = 'sum'
agg_dict['dt'] = 'min'

In [ ]:
online_sessions = online_expanded.groupby(['custno', 'session'], as_index=False).agg(agg_dict).copy()

online_sessions['start_time'] = online_sessions['dt']
del(online_sessions['dt'])

In [ ]:
online_sessions.head().T

## Create Features

In [ ]:
orders_and_dates = modeling_dataset_orders[['ordno','custno','orderdate']]

In [ ]:
orders_and_sessions = orders_and_dates.merge(online_sessions, on='custno').copy()

orders_and_sessions = orders_and_sessions[
    orders_and_sessions['orderdate']>orders_and_sessions['start_time']
]

In [ ]:
orders_and_sessions['days_before'] = (
    orders_and_sessions['orderdate'] - orders_and_sessions['start_time']
).astype('timedelta64[s]')/3600/24

In [ ]:
orders_and_sessions.columns.values

In [ ]:
orders_and_sessions.head(5).T

In [ ]:
orders_and_sessions_clean = orders_and_sessions[
    [
        'ordno','days_before',
        'E1:-1.0', 'E1:1.0', 'E1:2.0', 'E1:4.0', 'E1:5.0', 'E1:6.0',
        'E1:7.0', 'E1:8.0', 'E1:9.0', 'E1:10.0', 'E1:11.0',
        
        'E2:1', 'E2:2', 'E2:3','E2:4', 'E2:5',
        'E2:6', 'E2:7', 'E2:8', 'E2:9', 'E2:10', 
        
        'Cat:1', 'Cat:2', 'Cat:3', 
    ]
]

In [ ]:
def get_session_event_sum(all_previous_sessions, min_b4, max_b4):
    relevant_previous_sessions = all_previous_sessions[
        (all_previous_sessions['days_before']>min_b4) 
        & (all_previous_sessions['days_before']< max_b4)
    ]
    del(relevant_previous_sessions['days_before'])
    
    session_event_sum = relevant_previous_sessions.groupby('ordno', as_index=False).sum()
    
    # Fix Column Names
    agg_cols = list(session_event_sum.columns)[1:]
    new_cols = [x+f'_{min_b4}_{max_b4}' for x in agg_cols]
    new_cols = ['ordno'] + new_cols
    session_event_sum.columns = new_cols

    return session_event_sum

In [ ]:
session_event_sum_0_30 = get_session_event_sum(orders_and_sessions_clean,0,30)
session_event_sum_30_350 = get_session_event_sum(orders_and_sessions_clean,30,350)
session_event_sum_350_380 = get_session_event_sum(orders_and_sessions_clean,350,380)

In [ ]:
session_event_sum_0_30

In [ ]:
assert len(set(session_event_sum_0_30['ordno'])) == session_event_sum_0_30.shape[0]
assert len(set(session_event_sum_30_350['ordno'])) == session_event_sum_30_350.shape[0]
assert len(set(session_event_sum_350_380['ordno'])) == session_event_sum_350_380.shape[0]

assert len(set(modeling_dataset['ordno'])) == modeling_dataset.shape[0]

In [ ]:
modeling_dataset_full = modeling_dataset_orders.merge(
    session_event_sum_0_30,
    on='ordno',
    how='left'
).merge(
    session_event_sum_30_350,
    on='ordno',
    how='left'
).merge(
    session_event_sum_350_380,
    on='ordno',
    how='left'
)

In [ ]:
modeling_dataset_full.shape[0]

In [ ]:
modeling_dataset.shape[0]

In [ ]:
assert modeling_dataset_full.shape[0] == modeling_dataset.shape[0]

In [ ]:
assert modeling_dataset_full.shape[0] == order_totals.shape[0]

In [ ]:
assert len(set(modeling_dataset_full['ordno'])) == modeling_dataset_full.shape[0]

# Clean Up Final Modeling Dataset

#### Drop Redundant Variables

In [ ]:
drop_vars = [
    'ordno_current_x', 'ordno_current_y', 'ordno_current'
]

In [ ]:
modeling_dataset_final = modeling_dataset_full.drop(drop_vars, axis=1)

#### Filling Missing Counts with 0s

In [ ]:
modeling_dataset_final = modeling_dataset_final.fillna(0).copy()

# Split into Training and Test

In [ ]:
sum(modeling_dataset_final['orderdate']>pd.to_datetime('2018-12-01'))/modeling_dataset_final.shape[0]

# Create Features

In [ ]:
online.shape

In [ ]:
custno_join[
    (custno_join['orderdate']>custno_join['dt'])
].shape

In [ ]:
online = pd.read_csv('data/online.csv')
online['dt'] = pd.to_datetime(online['dt'])

In [ ]:
order = pd.read_csv('data/order.csv')
order['orderdate'] = pd.to_datetime(order['orderdate'])

In [ ]:
online.head()

In [ ]:
order.head()

In [ ]:
custno_join = pd.merge(order,online,how='left',on=['custno'])